In [1]:
import requests
from fake_useragent import UserAgent
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup
from IPython.display import clear_output
from fake_useragent import UserAgent
import json
from collections import Counter

import time
import datetime
import pandas as pd
import pickle

import pymorphy3

from tqdm import tqdm
from joblib import Parallel, delayed

import json

# На вход подается pickle-файл с последним обновлением, на выходе получаем обновленные csv-файлы для каждого макропоказателя

In [2]:
dict_keys = {

'export_import':[['экспорт', 'импорт'],1],
  
'infl':[['инфляция', 'цена', 'цены', 'рост'],2],
    
'tov_oborot':[['розничный', 'товарооборот', 
                        'продукция', 'розница', 'товар', 'услуга', 'производство', 'торговля'],2],
    
'invest':[['инвестиции', 'капитал', 'основной', 'пии', 'валовые', 'привлечение'],2],   
    
'bezrab':[['безработица', 'занятость', 'кадров', 'кадры', 'дефицит'],2],  

'real_den_doh':[['реальные', 'располагаемые', 'денежные', 'доходы', 'население', 'бедность',
             'прожиточный', 'минимум', 'индексация'],3],

'real_zp':[['реальная', 'заработная', 'плата', 'денежные', 'доходы', 'население',
             'бедность', 'прожиточный', 'минимум', 'индексация'],3],
    
'base_econom': [['ставка', 'экономика', 'инфляция', 'уровень', 'снижение', 'экономический', 'прогноз', 'ключевой', 
                'глава', 'считать', 'регулятор', 'повышение', 'заявить', 'сказать', 'решение', 'ситуация', 'оценка', 
                'снижение', 'активность', 'рецессия', 'процентная', 'процент', 'безработица', 'стагнация', 'спад', 'замедлить',
                'замедлиться', 'замедляться', 'износ', 'перенос', 'остановка', 'приостановка', 'остановить', 'негатив',
                'негативный', 'замедление'], 4],

'prom_proiz': [['промышленность', 'добыча', 'ископаемый', 'производством', 'обрабатывающим', 'электричеству', 
                'газ', 'энергия', 'вода', 'товар', 'услуга'], 3]}

In [3]:
m = pymorphy3.MorphAnalyzer()

def pymorphy_lemma(text='The text must be passed to the function as a string!'):
    text = re.sub(r'[^а-яёА-Я]+', ' ', text)
    tokens = text.lower().split()   
    lemmas = []
    
    for word in tokens:
        try:
            lemma = m.parse(word)[0].normal_form
            lemmas.append(lemma)
        except:
            continue
    return ', '.join(lemmas)

In [4]:
# функция поиска вхождения слов в тексте
# по одному слову плохо, т.к. может быть случаное совпадение в 
# составе другого слова косвенное упоминание, поэтому нужно 3-5
def find_top(x, det_wrds, intrs):
    intersect = len(set(det_wrds)&set(x.split(', ')))
    if intersect == intrs:
        return 1
    else:
        return 0

# CSV для каждого показателя

In [9]:
def make_sp_csv(key_name):

    key_words = dict_keys[key_name][0]
    prep_tag = pymorphy_lemma(' '.join(key_words))
    key_words = prep_tag.split(', ')

    drop_categ = ['Международная панорама', 'Биографии и справки', 'Космос']

    path = 'tass/tass_lemmas/'
    read_data = []
    read_dt = []
    i = 0
    with open(path + "lemma_tass_update_2024-07-12.pickle", 'rb') as file:               # файл с последними новостями (леммы)
        while True:
            try:
                item = pickle.load(file)
                read_dt.append(item['date'])
                i+=1
                if i%1000==0:
                    clear_output(wait=True)
                    print(i)

                if find_top(item['text'], key_words, dict_keys[key_name][1])==1 and item['category'] not in drop_categ:  
                    read_data.append(item)                
            except EOFError:
                break    
    file.close()
    pd.DataFrame(read_data).to_csv(f'lemmas/main_lemma_{key_name}.csv', mode = 'a', header=None, index=None)


In [8]:
%%time
for key_name in dict_keys.keys():
    make_sp_csv(key_name)

9000
CPU times: total: 5.69 s
Wall time: 8.62 s
